<a href="https://colab.research.google.com/github/TechWithEva/Telecom-X-Challenge2-G9/blob/main/TELECOM_X_Challenge2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TELECOM X - Churn de Clientes
---
#### ***Problema:***  La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.
---



In [1566]:
import pandas as pd

## Integración de Orígenes de Datos
< Extracción >

In [1567]:
url = ('https://raw.githubusercontent.com/TechWithEva/Telecom-X-Challenge2-G9/refs/heads/main/rawdata/TelecomX_Data.json')
base_ClientesTX = pd.read_json(url)
base_ClientesTX.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


## Tratamiento de Datos

#### Convertir a Archivo Plano

In [1568]:
baseNivel1 = base_ClientesTX[['customerID', 'Churn']].copy()

columnasAnidadas = ['customer', 'phone', 'internet', 'account']
for col in columnasAnidadas:
    baseNormalizada = pd.json_normalize(base_ClientesTX[col])
    baseNivel1 = pd.concat([baseNivel1, baseNormalizada], axis=1)
base_ClientesTX = baseNivel1

columnasNoBool=['customerID', 'Contract','tenure','PaymentMethod' ,'Charges.Monthly','Charges.Total','gender']
columnasBool = base_ClientesTX.columns[~base_ClientesTX.columns.isin(columnasNoBool)].to_list()
base_ClientesTX = base_ClientesTX.reindex(columns=pd.concat([pd.Series(columnasNoBool), pd.Series(columnasBool)]))
base_ClientesTX.insert(9, 'AutomaticPaymenthMethod', base_ClientesTX['PaymentMethod'].str.contains('automatic').astype(bool))
base_ClientesTX.insert(10, 'InternetServiceType', base_ClientesTX['InternetService'].apply(lambda x: None if x=='No' else x).astype(object))
base_ClientesTX['InternetService']=base_ClientesTX['InternetService'].apply(lambda x: False if x=='No' else True).astype(bool)
columnasNoBool=['customerID', 'Contract','tenure','PaymentMethod' ,'Charges.Monthly','Charges.Total','gender','InternetServiceType']
#

#### Tratamiento nulos y NaN

In [1569]:
#base_ClientesTX.info()
nulos = base_ClientesTX.isnull().sum()
#print(nulos)
nan = base_ClientesTX.isna().sum()
#print(nan)

#### Tratamiento Datos No Esperados y Tipado

In [1570]:
#reemplazar valores en atributos candidatos a ser booleanos
base_ClientesTX[columnasBool] = base_ClientesTX[columnasBool].replace({'No internet service': 'No','No phone service':'No','':'No'})
base_ClientesTX['SeniorCitizen'] = base_ClientesTX['SeniorCitizen'].replace({1:True, 0:False})
base_ClientesTX[columnasBool] = base_ClientesTX[columnasBool].replace({'Yes': True, 'No': False})

#tipar valores númericos
base_ClientesTX['tenure']= base_ClientesTX['tenure'].astype(int)
base_ClientesTX['Charges.Total']= base_ClientesTX['Charges.Total'].replace(' ','0.0').astype(float)

#remover espacios y estandarizar formato
columnasStr = base_ClientesTX[columnasNoBool].select_dtypes(include=['object'])
columnasStr
base_ClientesTX[columnasStr.columns] = columnasStr.apply(lambda x: x.str.strip().str.upper())
base_ClientesTX.sample(10)#InternetService
base_ClientesTX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customerID               7267 non-null   object 
 1   Contract                 7267 non-null   object 
 2   tenure                   7267 non-null   int64  
 3   PaymentMethod            7267 non-null   object 
 4   Charges.Monthly          7267 non-null   float64
 5   Charges.Total            7267 non-null   float64
 6   gender                   7267 non-null   object 
 7   Churn                    7267 non-null   bool   
 8   SeniorCitizen            7267 non-null   bool   
 9   AutomaticPaymenthMethod  7267 non-null   bool   
 10  InternetServiceType      5686 non-null   object 
 11  Partner                  7267 non-null   bool   
 12  Dependents               7267 non-null   bool   
 13  PhoneService             7267 non-null   bool   
 14  MultipleLines           

/tmp/ipython-input-1792211931.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base_ClientesTX['SeniorCitizen'] = base_ClientesTX['SeniorCitizen'].replace({1:True, 0:False})
/tmp/ipython-input-1792211931.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base_ClientesTX[columnasBool] = base_ClientesTX[columnasBool].replace({'Yes': True, 'No': False})


## Análisis de Datos

## Viz

## Conclusiones y Recomendaciones